In [0]:
!pip uninstall -y kaggle
!pip install kaggle==1.5.6
%env KAGGLE_USERNAME = muhammadamr
%env KAGGLE_KEY = 32b09959276a1bd786b9bebc09a18717
!kaggle competitions download -c fcis-sc-deeplearning-competition
!unzip '/content/fcis-sc-deeplearning-competition.zip'

In [0]:
import os 
import csv
import numpy as np
from tqdm import tqdm
from pandas import DataFrame as pd
from sklearn.model_selection import train_test_split
from keras import models
from keras import layers
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input as p_vgg16
from keras.applications.vgg16 import VGG16
from keras.applications.xception import preprocess_input as p_xception
from keras.applications.xception import Xception
from keras.applications.inception_resnet_v2 import preprocess_input as p_incep
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.applications.nasnet import preprocess_input as p_nasnet
from keras.applications.nasnet import NASNetLarge

In [0]:
dic = {'airport_inside': 1, 'bakery': 2, 'bedroom': 3, 'greenhouse': 4, 'gym': 5, 'kitchen': 6, 'operating_room': 7, 'poolinside': 8,'restaurant': 9, 'toystore': 10 }
datagen = ImageDataGenerator(
    width_shift_range = 0.3,
    height_shift_range=0.3,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
    )
IMG_SIZE_vgg16 = 224
IMG_SIZE_xception = 299
IMG_SIZE_incep = 299
IMG_SIZE_nasnet = 331

In [0]:
def loadImage(path, IMG_SIZE, preprocess):
    image = load_img(path, target_size=(IMG_SIZE, IMG_SIZE))
    image = img_to_array(image)
    image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
    image = preprocess(image)

    return image.reshape((IMG_SIZE, IMG_SIZE, 3))

In [0]:
def get_data(IMG_SIZE, preprocess, istot = False):
  Xtrain, Ytrain, Xtest, Ytest = [], [], [], []
  dir = '/content/train/train'
  for file in os.listdir(dir):
        filename = os.fsdecode(file)
        d = os.path.join(dir, filename)
        l, y = [], []
        for imag in tqdm(os.listdir(d)):
          imag_name = os.fsdecode(imag)
          path = os.path.join(d, imag_name)
          img = loadImage(path, IMG_SIZE, preprocess)
          l.append(img)
          tmp = [0.0]*10
          tmp[dic[filename]-1]=1.0;
          y.append(tmp)
        flw = datagen.flow(np.array(l), batch_size=1)
        while len(l)<300:
            cur = flw.next()
            l.append(cur[0])
            y.append(tmp)
        if istot:
          Xtrain += l
          Ytrain += y
        else :
          x_train, x_test, y_train, y_test  = train_test_split(l, y,test_size=0.2, shuffle=True)
          Xtrain += x_train
          Xtest += x_test
          Ytrain += y_train
          Ytest += y_test
  return np.array([i for i in Xtrain]), np.array([i for i in Ytrain]), np.array([i for i in Xtest]), np.array([i for i in Ytest])

In [0]:
def get_testData(IMG_SIZE, preprocess):
  dir = '/content/test/test'
  ID ,images = [],[]
  for file in tqdm(os.listdir(dir)):
        filename = os.fsdecode(file)
        d = os.path.join(dir, filename)
        img = loadImage(d, IMG_SIZE, preprocess)
        if img is not None:
          ID.append(filename)
          images.append(img)
  return ID, np.array(images)

In [0]:
def get_model(modl, IMG_SIZE):
  vgg_conv = modl(weights='imagenet', include_top=True, input_shape=(IMG_SIZE, IMG_SIZE,3))
  last = vgg_conv.get_layer(vgg_conv.layers[-2].name).output
  out = layers.Dense(10, activation='softmax')(last)
  model = models.Model(inputs=vgg_conv.input, outputs=out)
  # Freeze the layers except the last layer
  for layer in model.layers[:-1]:
      layer.trainable = False
  adam = optimizers.adam(0.001, amsgrad=True)
  model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['acc'])
  return model

In [0]:
def get_prob(predect, expected):
  cnt = [0]*10
  tot = [0]*10
  for i in range(len(predect)):
    tot[np.argmax(expected[i])] += 1
    if np.argmax(predect[i]) == np.argmax(expected[i]):
      cnt[np.argmax(predect[i])] += 1
  for i in range(10):
    cnt[i] /= tot[i]
  return cnt

In [0]:
def get_weight(prob):
  w = []
  tot = [0.0]*10
  for j in range(len(prob)):
    for i in range(10):
      tot[i] += prob[j][i]
  for j in range(len(prob)):
    tmp = [0.0]*10
    for i in range(10):
      tmp[i] = prob[j][i] / tot[i]
    w.append(tmp)
  return w

In [0]:
def getIdx(pred, w):
  l = []
  for i in range(10):
    sum = 0
    for j in range(len(pred)):
      sum += pred[j][i] * w[j][i]
    l.append(sum)
  return np.argmax(np.array(l))

In [0]:
# vgg16 model
X_train ,Y_train ,X_test ,Y_test = get_data(IMG_SIZE_vgg16, p_vgg16)
model1 = get_model(VGG16, IMG_SIZE_vgg16)
print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)

In [0]:
# fiting vgg16
model1.fit(X_train, Y_train, epochs=10, validation_data=(X_test,Y_test), shuffle=1, batch_size=32)
# model.fit(X_train, Y_train, epochs=2, shuffle=1)

In [0]:
# saving data for further need
X_train ,Y_train ,X_test ,Y_test = None, None, None, None
test, expected_vgg16, tmp, tmp = get_data(IMG_SIZE_vgg16, p_vgg16, True)
# predict all data for some calulation
predect_vgg16 = model1.predict(test)
test = None
print(predect_vgg16.shape)
print(expected_vgg16.shape)

In [0]:
# inception model
X_train ,Y_train ,X_test ,Y_test = get_data(IMG_SIZE_incep, p_incep)
model2 = get_model(InceptionResNetV2, IMG_SIZE_incep)
print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)

In [0]:
# fiting inception
model2.fit(X_train, Y_train, epochs=3, validation_data=(X_test,Y_test), shuffle=1, batch_size=32)
# model.fit(X_train, Y_train, epochs=2, shuffle=1)

In [0]:
# saving data for further need
X_train ,Y_train ,X_test ,Y_test = None, None, None, None
test, expected_incep, tmp, tmp = get_data(IMG_SIZE_incep, p_incep, True)
# predict all data for some calulation
predect_incep = model2.predict(test)
test = None
print(predect_incep.shape)
print(expected_incep.shape)

In [0]:
# nasnet model
X_train ,Y_train ,X_test ,Y_test = get_data(IMG_SIZE_nasnet, p_nasnet)
model3 = get_model(NASNetLarge, IMG_SIZE_nasnet)
print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)

In [0]:
# fiting nasnet
model3.fit(X_train, Y_train, epochs=10, validation_data=(X_test,Y_test), shuffle=1, batch_size=64)
# model.fit(X_train, Y_train, epochs=2, shuffle=1)

In [0]:
# saving data for further need
X_train ,Y_train ,X_test ,Y_test = None, None, None, None
test, expected_nasnet, tmp, tmp = get_data(IMG_SIZE_nasnet, p_nasnet, True)
# predict all data for some calulation
predect_nasnet = model3.predict(test)
test = None
print(predect_nasnet.shape)
print(expected_nasnet.shape)

In [0]:
# xception model
X_train ,Y_train ,X_test ,Y_test = get_data(IMG_SIZE_xception, p_xception)
model4 = get_model(Xception, IMG_SIZE_xception)
print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)

In [0]:
# fiting inception
model4.fit(X_train, Y_train, epochs=10, validation_data=(X_test,Y_test), shuffle=1, batch_size=64)
# model.fit(X_train, Y_train, epochs=2, shuffle=1)

In [0]:
# saving data for further need
X_train ,Y_train ,X_test ,Y_test = None, None, None, None
test, expected_xception, tmp, tmp = get_data(IMG_SIZE_xception, p_xception, True)
# predict all data for some calulation
predect_xception = model4.predict(test)
test = None
print(predect_xception.shape)
print(expected_xception.shape)

In [0]:
prob1 = get_prob(predect_vgg16, expected_vgg16)
prob2 = get_prob(predect_incep, expected_incep)
prob3 = get_prob(predect_nasnet, expected_nasnet)
prob4 = get_prob(predect_xception, expected_xception)
w = get_weight([prob1, prob2, prob3, prob4])

In [0]:
w = get_weight([prob1, prob3, prob4])

In [0]:
X_train ,Y_train ,X_test ,Y_test = None,None, None, None
test = None
print(w[0])
print(w[1])
print(w[2])
print(w[3])

In [0]:
#loding test
ID, test = get_testData(IMG_SIZE_vgg16, p_vgg16)
predect_vgg16 = model1.predict(test)
ID, test = get_testData(IMG_SIZE_incep, p_incep)
predect_incep = model2.predict(test)
ID, test = get_testData(IMG_SIZE_nasnet, p_nasnet)
predect_nasnet = model3.predict(test)
ID, test = get_testData(IMG_SIZE_xception, p_xception)
predect_xception = model4.predict(test)

In [0]:
file = []
for i in range(772):
  # idx = getIdx([predect_vgg16[i], predect_incep[i], predect_nasnet[i], predect_xception[i]],w)
  idx = getIdx([predect_vgg16[i], predect_nasnet[i], predect_xception[i]],w)
  label = idx + 1
  file.append([ID[i], label])

# file.append(['1412_mb_file_0a8c5_gif.jpg', '4'])

pd(np.array(file), columns=['Id', 'Label']).to_csv('submit.csv', index = False)


In [0]:
model1.save('vgg16.h5')
model2.save('inceptionResnet.h5')
model3.save('nasnet.h5')
model4.save('Xception.h5')